# Engenharia de Features e Modelos de M.L
#

## Configuração Inicial

In [0]:
%pip install mlflow>=3.0 --upgrade
%pip install lightgbm xgboost catboost scikit-plot imbalanced-learn synapseml
%pip install lightgbm
dbutils.library.restartPython()

In [0]:
import mlflow
import mlflow.spark
import mlflow.sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    precision_recall_curve, roc_curve, auc, classification_report,
    confusion_matrix, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.calibration import CalibratedClassifierCV
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from pyspark.sql import functions as F
from pyspark.sql.functions import col, sin, cos, sqrt, log, abs, pi, when
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler, RobustScaler
from pyspark.ml import Pipeline
from mlflow.models import infer_signature
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [0]:
mlflow.set_experiment("/Users/2106144@aluno.univesp.br/creditcard-fraud-detection")

In [0]:
train_set = spark.read.table("my_catalog.default.creditcard_treino_tcc")
test_set = spark.read.table("my_catalog.default.creditcard_teste_tcc")

print(f"Treino: {train_set.count():,} linhas, {len(train_set.columns)} colunas")
print(f"Teste: {test_set.count():,} linhas, {len(test_set.columns)} colunas")
train_set.printSchema()

## Engenharia de features

In [0]:
def create_features(df):
    """
    cria features derivadas baseadas na análise exploratória.
    """
    # features temporais
    df = df.withColumn("hour_of_day", (col("Time") / 3600).cast("int") % 24)
    df = df.withColumn("hour_sin", sin(2 * pi() * col("hour_of_day") / 24))
    df = df.withColumn("hour_cos", cos(2 * 3.14159 * col("hour_of_day") / 24))

    # interações baseadas em análise de correlação
    df = df.withColumn("V12_V14_interaction", col("V12") * col("V14"))
    df = df.withColumn("V3_V10_interaction", col("V3") * col("V10"))
    df = df.withColumn("V1_V2_complex", sqrt(col("V1")**2 + col("V2")**2))

    # features estatísticas avançadas (PCA magnitude)
    pca_cols = ['V1', 'V2', 'V3', 'V4', 'V7', 'V10', 'V11', 'V12', 'V14', 'V16', 'V17', 'V18']
    squared_expr = None
    for c in pca_cols:
        if squared_expr is None:
            squared_expr = col(c) * col(c)
        else:
            squared_expr = squared_expr + (col(c) * col(c))
    df = df.withColumn("pca_magnitude", sqrt(squared_expr))

    # transformações não-lineares
    df = df.withColumn("Amount_log", log(col("Amount") + 1))
    df = df.withColumn("Amount_sqrt", sqrt(col("Amount") + 0.1))
    df = df.withColumn("amount_to_mean_ratio", col("Amount") / 88.35)  # média do dataset
    df = df.withColumn("v1_anomaly", abs(col("V1") - (-0.001)) / 1.96)  # baseado na distribuição

    return df

In [0]:
# aplica engenharia de features
train_set_fe = create_features(train_set)
test_set_fe = create_features(test_set)

In [0]:
# salva os DataFrames com features no Unity Catalog
train_set_fe.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_treino_fe_tcc")
test_set_fe.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_teste_fe_tcc")

### Features selecionadas

In [0]:
final_features = [
    'V1', 'V2', 'V3', 'V4', 'V7', 'V10', 'V11', 'V12', 'V14', 'V16', 'V17', 'V18',
    'hour_sin', 'hour_cos', 'V12_V14_interaction', 'V3_V10_interaction',
    'V1_V2_complex', 'pca_magnitude', 'Amount_log', 'Amount_sqrt',
    'amount_to_mean_ratio', 'v1_anomaly'
]
target_column = 'Class'

print(f"Total de features: {len(final_features)}")
print("Features selecionadas:")
for i, feature in enumerate(final_features, 1):
    print(f"{i:2d}. {feature}")

## Balanceamento com pesos de classes

In [0]:
def calculate_class_weights(df, target_col):
    class_counts_df = df.groupBy(target_col).count()
    class_counts_list = class_counts_df.collect()
    counts_dict = {row[target_col]: row['count'] for row in class_counts_list}
    total = sum(counts_dict.values())
    num_classes = len(counts_dict)
    balanced_weights = {class_val: total / (num_classes * count) for class_val, count in counts_dict.items()}
    return balanced_weights

balanced_weights = calculate_class_weights(train_set_fe, 'Class')
train_set_weighted = train_set_fe.withColumn(
    "class_weight",
    when(col("Class") == 1, balanced_weights[1]).otherwise(balanced_weights[0])
)

print(f"Pesos de classe aplicados: {balanced_weights}")

In [0]:
train_set_weighted.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_treino_weighted_tcc")

## Pré-processamento com RobustScaler

In [0]:
assembler = VectorAssembler(inputCols=final_features, outputCol="features_raw")
scaler = RobustScaler(inputCol="features_raw", outputCol="features")
preprocessing_pipeline = Pipeline(stages=[assembler, scaler])

preprocessing_model = preprocessing_pipeline.fit(train_set_weighted)
train_processed = preprocessing_model.transform(train_set_weighted)
test_processed = preprocessing_model.transform(test_set_fe)

In [0]:
train_processed.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_treino_processed_tcc")
test_processed.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_teste_processed_tcc")

In [0]:
train_pd = train_processed.select(*final_features, "Class", "class_weight").toPandas()
test_pd = test_processed.select(*final_features, "Class").toPandas()

X_train = train_pd[final_features]
y_train = train_pd['Class']
sample_weights = train_pd['class_weight']
X_test = test_pd[final_features]
y_test = test_pd['Class']

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test) 

In [0]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=final_features)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=final_features)
X_train_df = pd.DataFrame(X_train, columns=final_features)
X_test_df = pd.DataFrame(X_test, columns=final_features)
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)

X_train_scaled_sdf = spark.createDataFrame(X_train_scaled_df)
X_test_scaled_sdf = spark.createDataFrame(X_test_scaled_df)
train_sdf = spark.createDataFrame(pd.concat([X_train_df, y_train_df], axis=1))
test_sdf = spark.createDataFrame(pd.concat([X_test_df, y_test_df], axis=1))

X_train_scaled_sdf.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_x_train_scaled_tcc")
X_test_scaled_sdf.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_x_test_scaled_tcc")
train_sdf.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_train_pdd_tcc")
test_sdf.write.mode("overwrite").saveAsTable("my_catalog.default.creditcard_test_pd_tcc")

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## Teste n1

In [0]:
linear_models = {
    "LogisticRegression": LogisticRegression(class_weight='balanced', random_state=42, max_iter=200),
    "NaiveBayes": GaussianNB()
}

tree_models = {
    "DecisionTree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "RandomForest": RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=200)
}

gradient_boosting_models = {
    "LightGBM": LGBMClassifier(class_weight='balanced', random_state=42),
    "XGBoost": XGBClassifier(scale_pos_weight=balanced_weights[1]/balanced_weights[0], random_state=42),
    "CatBoost": CatBoostClassifier(verbose=False, random_state=42),
}

neural_network_models = {
    "MLP_Classifier": MLPClassifier(random_state=42, early_stopping=True)
}

models = {**linear_models, **tree_models, **gradient_boosting_models, **neural_network_models}

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:470)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:768)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:80)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:348)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:59)
	at com.databricks.logging.AttributionContext$.withValue(Attr

### Treinamento dos modelos

In [0]:
results = {}

def train_and_evaluate(models_dict, use_sample_weight=True):
    for model_name, model in models_dict.items():
        with mlflow.start_run(run_name=model_name):
            try:
                
                if model_name in ["NaiveBayes", "MLP_Classifier"]:
                    model.fit(X_train_scaled, y_train)
                elif use_sample_weight:
                    model.fit(X_train_scaled, y_train, sample_weight=sample_weights)
                else:
                    model.fit(X_train_scaled, y_train)

                
                y_pred = model.predict(X_test_scaled)
                if hasattr(model, "predict_proba"):
                    y_prob = model.predict_proba(X_test_scaled)[:, 1]
                else:
                    y_prob = y_pred

                
                roc_auc = roc_auc_score(y_test, y_prob)
                avg_precision = average_precision_score(y_test, y_prob)
                precision = precision_score(y_test, y_pred, zero_division=0)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

                
                results[model_name] = {
                    'roc_auc': roc_auc,
                    'avg_precision': avg_precision,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1,
                    'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
                }

                
                mlflow.log_metrics({
                    'roc_auc': roc_auc,
                    'avg_precision': avg_precision,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1
                })

                
                mlflow.sklearn.log_model(
                    sk_model=model,
                    name=model_name,
                    input_example=X_test_scaled[:5]
                )

                print(f"Modelo {model_name} treinado, avaliado e salvo com sucesso.")

            except Exception as e:
                print(f"Erro ao treinar {model_name}: {str(e)}")
                mlflow.log_param("error", str(e))

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:470)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:768)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:80)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:348)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:59)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
train_and_evaluate(linear_models, use_sample_weight=False)

train_and_evaluate(tree_models)

train_and_evaluate(gradient_boosting_models)

train_and_evaluate({"MLP_Classifier": neural_network_models["MLP_Classifier"]}, use_sample_weight=False)

In [0]:
comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df.sort_values('roc_auc', ascending=False)
print(comparison_df[['roc_auc', 'avg_precision', 'precision', 'recall', 'f1_score']])

 Modelo              | ROC AUC   | Avg Precision | Precision  | Recal    | F1-Score  |
--------------------|-----------|---------------|------------|-----------|-----------|
 LogisticRegression | 0.976711  | 0.748399      | 0.052599   | 0.867347  | 0.099183  |
 MLP_Classifier     | 0.975406  | 0.744587      | 0.866667   | 0.795918  | 0.829787  |
 CatBoost           | 0.972523  | 0.785162      | 0.766990   | 0.806122  | 0.786070  |
 XGBoost            | 0.951283  | 0.807184      | 0.759615   | 0.806122  | 0.782178  |
 RandomForest       | 0.946974  | 0.801817      | 0.932432   | 0.704082  | 0.802326  |
 LightGBM           | 0.939870  | 0.754785      | 0.804348   | 0.755102  | 0.778947  |
 DecisionTree       | 0.862043  | 0.547695      | 0.755319   | 0.724490  | 0.739583  |


## _Fine-tuning_

### Ajuste Fino de Hiperparâmetros

In [0]:
params = {
    "LogisticRegression": {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    },
    "RandomForest": {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    },
    "XGBoost": {
        'n_estimators': [100, 200, 300],
        'max_depth': [6, 8, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    },
    "LightGBM": {
        'n_estimators': [100, 200, 300],
        'max_depth': [6, 8, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'num_leaves': [31, 50, 100]
    },
    "CatBoost": {
        'iterations': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'depth': [4, 6, 8]
    },
    "MLP_Classifier": {
        'hidden_layer_sizes': [(64,), (64, 32), (128, 64, 32)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01]
    }
}

In [0]:
stats_df = (
    train_set.groupBy("Class")
    .agg(
        F.round(F.expr("percentile_approx(Amount, 0.5)"), 2).alias("Mediana"),
        F.round(F.avg("Amount"), 2).alias("Média"),
        F.round(F.stddev("Amount"), 2).alias("Desvio Padrão"),
        F.round(F.stddev("Amount") / F.avg("Amount"), 2).alias("Coef. Variação")
    )
)

display(stats_df)

In [0]:
best_models = {}

for model_name, model in models.items():
    print(f"Ajustando hiperparâmetros para {model_name}...")
    if model_name in params:
        search = RandomizedSearchCV(model, params[model_name], cv=5, scoring='f1', n_iter=10, random_state=42)
        if model_name in ["NaiveBayes", "MLP_Classifier"]:
            search.fit(X_train_scaled, y_train)
        else:
            search.fit(X_train_scaled, y_train, sample_weight=sample_weights)
        best_models[model_name] = search.best_estimator_
        print(f"Melhores parâmetros para {model_name}: {search.best_params_}")
    else:
        best_models[model_name] = model

| Modelo              | Hiperparâmetros Otimizados                                                                                  |
|---------------------|-------------------------------------------------------------------------------------------------------------|
| Logistic Regression | C=100, penalty='l2', solver='liblinear'                                                                     |
| Random Forest       | n_estimators=100, max_depth=None, min_samples_split=2                                                       |
| XGBoost             | learning_rate=0.2, max_depth=8, n_estimators=300, subsample=0.9, colsample_bytree=1.0                       |
| LightGBM            | learning_rate=0.2, max_depth=8, n_estimators=300, num_leaves=50                                            |
| CatBoost            | learning_rate=0.1, depth=6, iterations=200                                                                  |
| MLP Classifier      | hidden_layer_sizes=(64, 32), activation='tanh', solver='adam', alpha=0.001                                 |


In [0]:
model_path = "models:/creditcard-fraud-detection/"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

common_requirements = [
    "scikit-learn==1.6.1",
    "cloudpickle==3.0.0",
    "xgboost",
    "lightgbm",
    "catboost"
]

for model_name, model in best_models.items():
    registered_model_name = f"creditcard-fraud-{model_name}-best"
    with mlflow.start_run(run_name=f"{model_name}_best_{timestamp}"):
        
        if model_name in params:
            mlflow.log_dict(params[model_name], "hyperparameters.json")
       
        if hasattr(model, "get_params"):
            mlflow.log_dict(model.get_params(), "best_params.json")
        
        X_sample = X_test_scaled[:5]
        y_sample = model.predict(X_sample)
        signature = infer_signature(X_sample, y_sample)
        mlflow.sklearn.log_model(
            sk_model=model,
            registered_model_name=registered_model_name,
            pip_requirements=common_requirements,
            signature=signature,
            input_example=X_sample
        )
        print(f"Melhor modelo {model_name} registrado no Unity Catalog como: {registered_model_name}")

### Calibração das Probabilidades

In [0]:
calibrated_models = {}

for model_name, model in best_models.items():
    if hasattr(model, "predict_proba"):
        print(f"Calibrando {model_name}...")
        calibrated_model = CalibratedClassifierCV(model, method='isotonic', cv=5)
        if model_name in ["NaiveBayes", "MLP_Classifier"]:
            calibrated_model.fit(X_train_scaled, y_train)
        else:
            calibrated_model.fit(X_train_scaled, y_train, sample_weight=sample_weights)
        calibrated_models[model_name] = calibrated_model
    else:
        calibrated_models[model_name] = model

In [0]:
model_path = "models:/creditcard-fraud-detection/"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

common_requirements = [
    "scikit-learn==1.6.1",
    "cloudpickle==3.0.0",
    "xgboost",
    "lightgbm",
    "catboost"
]

for model_name, model in calibrated_models.items():
    registered_model_name = f"creditcard-fraud-{model_name}-calibrated"
    
    X_sample = X_test_scaled[:5]
    y_sample = model.predict(X_sample)
    signature = infer_signature(X_sample, y_sample)
    mlflow.sklearn.log_model(
        sk_model=model,
        registered_model_name=registered_model_name,
        pip_requirements=common_requirements,
        signature=signature,
        input_example=X_sample
    )
    print(f"Modelo calibrado {model_name} registrado no Unity Catalog como: {registered_model_name}")

### Ajuste do Threshold de Decisão

In [0]:
best_thresholds = {}

for model_name, model in calibrated_models.items():
    if hasattr(model, "predict_proba"):
        print(f"Ajustando threshold para {model_name}...")
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
        best_threshold = thresholds[np.argmax(f1_scores)]
        best_thresholds[model_name] = best_threshold
        print(f"Melhor threshold para {model_name}: {best_threshold:.4f}")

| Modelo              | Melhor Threshold |
|---------------------|-----------------|
| LogisticRegression  | 0.9914          |
| NaiveBayes          | 0.1370          |
| DecisionTree        | 0.5339          |
| RandomForest        | 0.9871          |
| LightGBM            | 0.5024          |
| XGBoost             | 0.9958          |
| CatBoost            | 0.9955          |
| MLP_Classifier      | 0.3624          |

In [0]:
import mlflow
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
thresholds_artifact_name = f"best_thresholds_{timestamp}.json"

with mlflow.start_run(run_name=f"best_thresholds_{timestamp}"):
    mlflow.log_dict(best_thresholds, thresholds_artifact_name)
    mlflow.log_param("description", "Best thresholds for each model after calibration and optimization")
    print(f"Best thresholds saved to Unity Catalog as artifact: {thresholds_artifact_name}")

### Avaliação dos Modelos com os Novos Parâmetros e Thresholds

In [0]:
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, f1_score, confusion_matrix
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

results_2 = {}

for model_name, model in calibrated_models.items():
    print(f"Avaliando {model_name}...")

    try:
        with mlflow.start_run(run_name=f"{model_name}_optimized"):
            if model_name in best_thresholds:
                y_prob = model.predict_proba(X_test_scaled)[:, 1]
                y_pred = (y_prob >= best_thresholds[model_name]).astype(int)
            else:
                y_pred = model.predict(X_test_scaled)
                y_prob = y_pred if not hasattr(model, "predict_proba") else model.predict_proba(X_test_scaled)[:, 1]

            roc_auc = roc_auc_score(y_test, y_prob)
            avg_precision = average_precision_score(y_test, y_prob)
            precision = precision_score(y_test, y_pred, zero_division=0)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

            results_2[model_name] = {
                'roc_auc': roc_auc,
                'avg_precision': avg_precision,
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
            }

            
            mlflow.log_metrics({
                'roc_auc': roc_auc,
                'avg_precision': avg_precision,
                'precision': precision,
                'recall': recall,
                'f1_score': f1
            })

            
            X_sample = X_test_scaled[:5]
            y_sample = model.predict(X_sample)
            signature = infer_signature(X_sample, y_sample)
            mlflow.sklearn.log_model(
                sk_model=model,
                name=f"{model_name}_optimized_model",
                signature=signature,
                input_example=X_sample
            )

            print(f"Modelo {model_name} avaliado e salvo com sucesso.")

    except Exception as e:
        print(f"Erro ao avaliar {model_name}: {str(e)}")
        if mlflow.active_run():
            mlflow.end_run()
        with mlflow.start_run(run_name=f"{model_name}_optimized"):
            mlflow.log_param("error", str(e))

In [0]:
comparison_df = pd.DataFrame(results_2).T
comparison_df = comparison_df.sort_values('roc_auc', ascending=False)
print(comparison_df[['roc_auc', 'avg_precision', 'precision', 'recall', 'f1_score']])

| Modelo              | roc_auc   | avg_precision | precision  | recall    | f1_score  |
|---------------------|-----------|---------------|------------|-----------|-----------|
| CatBoost            | 0.978671  | 0.772139      | 0.925926   | 0.765306  | 0.837989  |
| MLP_Classifier      | 0.975384  | 0.765712      | 0.894118   | 0.775510  | 0.830601  |
| LogisticRegression  | 0.968285  | 0.735655      | 0.852273   | 0.765306  | 0.806452  |
| XGBoost             | 0.958456  | 0.814541      | 0.938272   | 0.775510  | 0.849162  |
| RandomForest        | 0.935793  | 0.799960      | 0.883721   | 0.775510  | 0.826087  |
| NaiveBayes          | 0.930234  | 0.118348      | 0.142086   | 0.806122  | 0.241590  |
| LightGBM            | 0.907849  | 0.789173      | 0.849462   | 0.806122  | 0.827225  |
| DecisionTree        | 0.897795  | 0.732142      | 0.883721   | 0.775510  | 0.826087  |


In [0]:
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

In [0]:
from imblearn.combine import SMOTETomek, SMOTEENN
smote_tomek = SMOTETomek(random_state=42)
X_train_smote_tomek, y_train_smote_tomek = smote_tomek.fit_resample(X_train_scaled, y_train)

In [0]:
from imblearn.over_sampling import ADASYN, BorderlineSMOTE
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_scaled, y_train)

In [0]:
def train_and_evaluate_with_balancing(X_train_balanced, y_train_balanced, balancing_name):
    results_smote_2 = {}
    for model_name, model in models.items():
        print(f"Avaliando {model_name} com {balancing_name}...")
        try:
            with mlflow.start_run(run_name=f"{model_name}_{balancing_name}"):
                
                model.fit(X_train_balanced, y_train_balanced)

                
                y_pred = model.predict(X_test_scaled)
                if hasattr(model, "predict_proba"):
                    y_prob = model.predict_proba(X_test_scaled)[:, 1]
                else:
                    y_prob = y_pred

                
                roc_auc = roc_auc_score(y_test, y_prob)
                avg_precision = average_precision_score(y_test, y_prob)
                precision = precision_score(y_test, y_pred, zero_division=0)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

                
                results_smote_2[model_name] = {
                    'roc_auc': roc_auc,
                    'avg_precision': avg_precision,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1,
                    'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
                }

                
                mlflow.log_metrics({
                    'roc_auc': roc_auc,
                    'avg_precision': avg_precision,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1
                })

                
                X_sample = X_test_scaled[:5]
                y_sample = model.predict(X_sample)
                signature = infer_signature(X_sample, y_sample)
                mlflow.sklearn.log_model(
                    sk_model=model,
                    name=f"{model_name}_{balancing_name}_model",
                    signature=signature,
                    input_example=X_sample
                )

                print(f"Modelo {model_name} treinado, avaliado e salvo com sucesso.")

        except Exception as e:
            print(f"Erro ao avaliar {model_name} com {balancing_name}: {str(e)}")
            if mlflow.active_run():
                mlflow.end_run()
            with mlflow.start_run(run_name=f"{model_name}_{balancing_name}"):
                mlflow.log_param("error", str(e))

    return results_smote_2


In [0]:
results_no_balancing = train_and_evaluate_with_balancing(X_train_scaled, y_train, "NoBalancing")

results_smote = train_and_evaluate_with_balancing(X_train_smote, y_train_smote, "SMOTE")

results_smote_tomek = train_and_evaluate_with_balancing(X_train_smote_tomek, y_train_smote_tomek, "SMOTETomek")

results_adasyn = train_and_evaluate_with_balancing(X_train_adasyn, y_train_adasyn, "ADASYN")

In [0]:
#import pandas as pd

df_no_balancing = pd.DataFrame(results_no_balancing).T
df_smote = pd.DataFrame(results_smote).T
df_smote_tomek = pd.DataFrame(results_smote_tomek).T
df_adasyn = pd.DataFrame(results_adasyn).T

df_no_balancing['balancing'] = 'NoBalancing'
df_smote['balancing'] = 'SMOTE'
df_smote_tomek['balancing'] = 'SMOTETomek'
df_adasyn['balancing'] = 'ADASYN'

df_comparison = pd.concat([df_no_balancing, df_smote, df_smote_tomek, df_adasyn])
df_comparison.reset_index(inplace=True)
df_comparison.rename(columns={'index': 'model'}, inplace=True)

cols = ['model', 'balancing', 'roc_auc', 'avg_precision', 'precision', 'recall', 'f1_score']
df_comparison = df_comparison[cols]

display(df_comparison.sort_values(['model', 'balancing']))

 Modelo              | Balanceamento    | roc_auc   | avg_precision | precision  | recall    | f1_score  |
--------------------|-------------|-----------|---------------|------------|-----------|-----------|
 CatBoost           | ADASYN      | 0.956640  | 0.792555      | 0.714286   | 0.765306  | 0.738916  |
 CatBoost           | SMOTE       | 0.952647  | 0.797711      | 0.806122   | 0.806122  | 0.806122  |
 CatBoost           | SMOTETomek  | 0.955001  | 0.800558      | 0.795918   | 0.795918  | 0.795918  |
 DecisionTree       | ADASYN      | 0.851223  | 0.300396      | 0.425926   | 0.704082  | 0.530769  |
 DecisionTree       | SMOTE       | 0.881967  | 0.375554      | 0.490196   | 0.765306  | 0.597610  |
 DecisionTree       | SMOTETomek  | 0.896957  | 0.323693      | 0.406250   | 0.795918  | 0.537931  |
 LightGBM           | ADASYN      | 0.969664  | 0.783541      | 0.678899   | 0.755102  | 0.714976  |
 LightGBM           | SMOTE       | 0.969848  | 0.783046      | 0.838710   | 0.795918  | 0.816754  |
 LightGBM           | SMOTETomek  | 0.962959  | 0.795505      | 0.747573   | 0.785714  | 0.766169  |
 LogisticRegression | ADASYN      | 0.935248  | 0.701639      | 0.017921   | 0.867347  | 0.035117  |
 LogisticRegression | SMOTE       | 0.975329  | 0.741450      | 0.051069   | 0.877551  | 0.096521  |
 LogisticRegression | SMOTETomek  | 0.976201  | 0.741972      | 0.050263   | 0.877551  | 0.095080  |
 MLP_Classifier     | ADASYN      | 0.964547  | 0.757001      | 0.658333   | 0.806122  | 0.724771  |
 MLP_Classifier     | SMOTE       | 0.958861  | 0.765172      | 0.641667   | 0.785714  | 0.706422  |
 MLP_Classifier     | SMOTETomek  | 0.949917  | 0.771810      | 0.696429   | 0.795918  | 0.742857  |
 NaiveBayes         | ADASYN      | 0.953121  | 0.099563      | 0.055785   | 0.826531  | 0.104516  |
 NaiveBayes         | SMOTE       | 0.951536  | 0.117288      | 0.076487   | 0.826531  | 0.140017  |
 NaiveBayes         | SMOTETomek  | 0.951352  | 0.117898      | 0.076923   | 0.826531  | 0.140747  |
 RandomForest       | ADASYN      | 0.964573  | 0.807835      | 0.883721   | 0.775510  | 0.826087  |
 RandomForest       | SMOTE       | 0.965979  | 0.810529      | 0.905882   | 0.785714  | 0.841530  |
 RandomForest       | SMOTETomek  | 0.969001  | 0.817386      | 0.865169   | 0.785714  | 0.823529  |
 XGBoost            | ADASYN      | 0.952709  | 0.797816      | 0.690265   | 0.795918  | 0.739336  |
 XGBoost            | SMOTE       | 0.956289  | 0.812904      | 0.696429   | 0.795918  | 0.742857  |
 XGBoost            | SMOTETomek  | 0.942088  | 0.800165      | 0.644628   | 0.795918  | 0.712329  |